In [1]:
import math
import numpy as np
import torch
import pkbar
from unet3d.config import *
from utils.Visualization import ImageSliceViewer3D
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
import glob
from torch.optim import Adam
from unet3d.dataset import SAIADDataset
from torch.utils.data import DataLoader
from patchify import patchify
import nrrd

%load_ext autoreload
%autoreload 2
%matplotlib inline
excl_patients = ['SAIAD 1','SAIAD 2','SAIAD 5','SAIAD 7','SAIAD 9','SAIAD 11','SAIAD 12','SAIAD 13','SAIAD 14',
                    'SAIAD 15','SAIAD 20 vasculaire', 'SAIAD 20 BIS vasculaire', 'SAIAD 19 BIS vasculaire']


In [2]:
dataset = SAIADDataset(excl_patients=excl_patients, batch_size=8, epochs=100, n_batches=100, load_data_to_memory=True)

Generating patients probabilities...


In [ ]:
ImageSliceViewer3D(np.array(dataset.patients_probabilities[3]))

In [ ]:
arr = np.zeros(dataset.patients_probabilities[0].shape)
for center in dataset.patients_centers[0]:
    arr[center[0], center[1], center[2]]+=1
ImageSliceViewer3D(arr)


In [ ]:
n=0
ImageSliceViewer3D(np.array(X_batch[n]), np.array(y_batch[n,:,:,:,1]))

In [5]:
# DataLoader test
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, pin_memory=True, num_workers=8)

In [14]:
for imgs, labels in tqdm(dataloader):
    pass

100%|██████████| 100/100 [00:14<00:00,  6.79it/s]


In [15]:
n=0
ImageSliceViewer3D(np.array(imgs[n,0]), np.array(labels[n,1,:,:,:]))

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [10]:
labels.shape

torch.Size([8, 5, 96, 96, 96])